In [ ]:
try:
    import AI4Water
except ModuleNotFoundError:
    !python -m pip install "AI4Water[all] @ git+https://github.com/AtrCheema/AI4Water.git@dev"

In [ ]:
import pandas as pd 

from AI4Water.utils.datasets import MtropicsLaos
from AI4Water.utils.resample import Resampler 
from AI4Water.experiments import TransformationExperiments

In [ ]:
laos = MtropicsLaos()

pcp = laos.fetch_pcp(st='20110101', en='20181231')
pcp = pcp.interpolate('linear', limit=5)
pcp = pcp.fillna(0.0)

In [ ]:
w = laos.fetch_weather_station_data("20110101", "20181231")

In [ ]:
ecoli = laos.fetch_ecoli(en="20181231")
ecoli = ecoli.dropna()
ecoli_6min = ecoli.resample('6min').mean()

In [ ]:
w = w.interpolate()
w = w.bfill()
assert int(w.isna().sum().sum()) == 0

In [ ]:
w.columns = ['temp', 'rel_hum', 'wind_speed', 'sol_rad']

w_6min = Resampler(w,
               freq='6min',
               how={'temp': 'linear',
                    'rel_hum': 'linear',
                    'wind_speed': 'linear',
                    'sol_rad':'linear'
                    }
               )()

In [ ]:
st, en = ecoli_6min.index[0], ecoli_6min.index[-1]
data = pd.concat([w_6min[st:en], pcp[st:en], ecoli_6min], axis=1)

In [ ]:
cases = {
    'model_None': {'transformation': None},
    'model_logn': {'transformation':{'method': 'log', 'replace_nans': True,
                              'replace_zeros': True,
                              'features': ['E-coli_4dilutions']}},
    'model_log10': {'transformation': {'method': 'log10', 'replace_nans': True,
                                     'replace_zeros': True,
                                     'features': ['E-coli_4dilutions']}},
    'model_robust': {'transformation': {'method': 'robust', 'features': ['E-coli_4dilutions']}},
    'model_zscore': {'transformation': {'method': 'zscore', 'features': ['E-coli_4dilutions']}},
    'model_minmax': {'transformation': {'method': 'minmax', 'features': ['E-coli_4dilutions']}},
    'model_quantile': {'transformation': {'method': 'quantile', 'features': ['E-coli_4dilutions']}},
}

In [ ]:
model_args = {
    'epochs': 500,
    'patience': 50,
    'test_fraction': 0.25,
    'val_data': 'same',
    'verbosity': 0
}

In [ ]:
class MyTransformationExperiment(TransformationExperiments):

    def update_paras(self, **suggested_paras):
        """This method must update all the parameters to build the dl4sq 'Model'."""
        return {
            'model': {
                'layers': {
                    "LSTM": {"units": suggested_paras.get('lstm_units', 64), "dropout": 0.2},
                    "Dense": 1
                }
            },
            'lookback': suggested_paras.get('lookback', 10),
            'batch_size': suggested_paras.get('batch_size', 8),
            'lr': suggested_paras.get('lr', 0.001),
            'transformation': suggested_paras['transformation']
        }

In [ ]:
experiment = MyTransformationExperiment(data=data,
                                        cases=cases,
                                        exp_name='ecoli_output_transformation',
                                        **model_args)

In [ ]:
experiment.fit(fit_kws={'indices': 'random'})

In [ ]:
experiment.compare_errors('r2')

In [ ]:
errs = experiment.compare_errors('nse')

In [ ]:
errs = experiment.compare_errors('rmse')

In [ ]:
errs = experiment.compare_errors('msle')

In [ ]:

errs = experiment.compare_errors('mape')

In [ ]:
experiment.plot_taylor(
             figsize=(7, 10),
             save=True,
             grid_kws={'axis': 'x', 'color': 'g', 'lw': 1.0},
            )